# Bayesian Change Point Analysis: Brent Oil Prices

## Task 2: Change Point Modeling and Insight Generation

This notebook implements Bayesian change point detection using PyMC to:
1. Build and fit a change point model
2. Identify structural breaks in oil prices
3. Quantify the impact of detected changes
4. Associate changes with major events

**Author:** Daniel Mituku  
**Date:** February 2026

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
az.style.use('arviz-darkgrid')

# Add src to path
import sys
sys.path.append('..')
from src.data_loader import load_brent_oil_data, load_events_data, add_derived_features, prepare_data_for_modeling
from src.analysis import (
    build_single_changepoint_model,
    build_changepoint_model_log_returns,
    sample_model,
    check_convergence,
    get_changepoint_date,
    quantify_impact,
    plot_posterior_tau,
    plot_price_with_changepoint
)

# Set random seed for reproducibility
np.random.seed(42)

print(f"PyMC version: {pm.__version__}")
print(f"ArviZ version: {az.__version__}")

## 1. Load Data

In [ ]:
# Load price data
df = load_brent_oil_data('../data/raw/brent_oil_prices.csv')
df = add_derived_features(df)

# Load events
events_df = load_events_data('../events/major_events.csv')

print(f"Data range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Total observations: {len(df)}")

In [ ]:
# For demonstration, let's focus on a specific period
# The 2008 Financial Crisis period (2007-2010)
start_date = '2007-01-01'
end_date = '2010-12-31'

prices, time_idx, dates = prepare_data_for_modeling(df, start_date, end_date)
print(f"Analysis period: {dates.iloc[0]} to {dates.iloc[-1]}")
print(f"Number of observations: {len(prices)}")

In [ ]:
# Visualize the subset
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, prices, 'b-', linewidth=1)
ax.set_xlabel('Date')
ax.set_ylabel('Price (USD/barrel)')
ax.set_title(f'Brent Oil Prices ({start_date} to {end_date})')

# Mark relevant events
for _, event in events_df.iterrows():
    if pd.to_datetime(start_date) <= event['date'] <= pd.to_datetime(end_date):
        ax.axvline(event['date'], color='red', linestyle='--', alpha=0.5)
        ax.text(event['date'], ax.get_ylim()[1] * 0.95, event['event_name'], 
                rotation=90, fontsize=8, va='top')

plt.tight_layout()
plt.show()

## 2. Build the Bayesian Change Point Model

### Model Specification

We model the prices as coming from two different normal distributions, with a change point τ:

$$y_t \sim \begin{cases} \mathcal{N}(\mu_1, \sigma_1) & \text{if } t < \tau \\ \mathcal{N}(\mu_2, \sigma_2) & \text{if } t \geq \tau \end{cases}$$

**Priors:**
- $\tau \sim \text{DiscreteUniform}(0, n-1)$
- $\mu_1, \mu_2 \sim \mathcal{N}(\bar{y}, 2 \cdot s_y)$
- $\sigma_1, \sigma_2 \sim \text{HalfNormal}(s_y)$

In [ ]:
# Build the model
model = build_single_changepoint_model(prices)

# Display model structure
print(model)

## 3. Run MCMC Sampling

In [ ]:
# Sample from the posterior
# Note: This may take a few minutes
trace = sample_model(model, draws=2000, tune=1000, chains=4, random_seed=42)

## 4. Check Convergence

In [ ]:
# Check convergence diagnostics
summary = check_convergence(trace)
print("\nConvergence Diagnostics:")
print(summary)

In [ ]:
# Trace plots
az.plot_trace(trace, var_names=['tau', 'mu_1', 'mu_2'])
plt.tight_layout()
plt.show()

In [ ]:
# Check r_hat values (should be close to 1.0)
print("\nR-hat values (should be < 1.05):")
for var in ['tau', 'mu_1', 'mu_2', 'sigma_1', 'sigma_2']:
    rhat = az.rhat(trace, var_names=[var])
    print(f"  {var}: {float(rhat[var].values):.4f}")

## 5. Identify the Change Point

In [ ]:
# Get change point date
tau_date, tau_lower, tau_upper = get_changepoint_date(trace, dates)

print(f"\nDetected Change Point:")
print(f"  Most Probable Date: {tau_date.strftime('%Y-%m-%d')}")
print(f"  95% Credible Interval: [{tau_lower.strftime('%Y-%m-%d')}, {tau_upper.strftime('%Y-%m-%d')}]")

In [ ]:
# Plot posterior distribution of tau
fig = plot_posterior_tau(trace, dates, events_df)
plt.show()

## 6. Quantify the Impact

In [ ]:
# Quantify the impact
impact = quantify_impact(trace)

print("\nImpact Quantification:")
print(f"  Average price BEFORE change point: ${impact['before_mean']:.2f}")
print(f"  Average price AFTER change point: ${impact['after_mean']:.2f}")
print(f"  Absolute change: ${impact['absolute_change']:.2f}")
print(f"  Percentage change: {impact['percent_change']:.1f}%")
print(f"  Probability of price increase: {impact['prob_increase']*100:.1f}%")

In [ ]:
# Plot posterior distributions for mu_1 and mu_2
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# mu_1 (before)
az.plot_posterior(trace, var_names=['mu_1'], ax=axes[0])
axes[0].set_title('Posterior: Mean Price Before Change Point')

# mu_2 (after)
az.plot_posterior(trace, var_names=['mu_2'], ax=axes[1])
axes[1].set_title('Posterior: Mean Price After Change Point')

plt.tight_layout()
plt.show()

In [ ]:
# Plot the difference distribution
mu_1_samples = trace.posterior['mu_1'].values.flatten()
mu_2_samples = trace.posterior['mu_2'].values.flatten()
diff = mu_2_samples - mu_1_samples

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(diff, bins=50, density=True, alpha=0.7, color='steelblue')
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='No Change')
ax.axvline(np.mean(diff), color='green', linestyle='-', linewidth=2, label=f'Mean: ${np.mean(diff):.2f}')
ax.set_xlabel('Price Difference (After - Before)')
ax.set_ylabel('Density')
ax.set_title('Posterior Distribution of Price Change')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nProbability that price decreased: {(diff < 0).mean()*100:.1f}%")

## 7. Associate Changes with Events

In [ ]:
# Find closest events to the detected change point
print(f"\nDetected change point: {tau_date.strftime('%Y-%m-%d')}")
print("\nNearby events:")

for _, event in events_df.iterrows():
    days_diff = (event['date'] - tau_date).days
    if abs(days_diff) <= 60:  # Within 60 days
        print(f"  {event['event_name']} ({event['date'].strftime('%Y-%m-%d')})")
        print(f"    - Days from change point: {days_diff}")
        print(f"    - Type: {event['event_type']}")
        print(f"    - Description: {event['description']}")
        print()

In [ ]:
# Plot price with change point and events
fig = plot_price_with_changepoint(prices, dates, trace, events_df)
plt.show()

## 8. Summary and Interpretation

### Key Findings

Based on our Bayesian change point analysis of Brent oil prices during 2007-2010:

1. **Detected Change Point**: The model identifies a significant structural break in oil prices around [DATE]

2. **Impact Quantification**:
   - Before the change point, average daily prices were approximately $X per barrel
   - After the change point, average daily prices shifted to approximately $Y per barrel
   - This represents a Z% change in price levels

3. **Event Association**:
   - The detected change point coincides with [EVENT NAME]
   - This suggests the [EVENT] may have triggered a fundamental shift in market dynamics

### Important Caveats

- **Correlation ≠ Causation**: Detecting a change point near an event does not prove the event caused the change
- **Multiple Factors**: Oil prices are influenced by numerous factors simultaneously
- **Model Limitations**: A single change point model simplifies the complex reality of market dynamics

In [ ]:
# Save results
results = {
    'analysis_period': f"{start_date} to {end_date}",
    'change_point_date': tau_date.strftime('%Y-%m-%d'),
    'change_point_ci_lower': tau_lower.strftime('%Y-%m-%d'),
    'change_point_ci_upper': tau_upper.strftime('%Y-%m-%d'),
    'mean_before': impact['before_mean'],
    'mean_after': impact['after_mean'],
    'percent_change': impact['percent_change'],
    'prob_increase': impact['prob_increase']
}

# Save to file
results_df = pd.DataFrame([results])
results_df.to_csv('../data/processed/changepoint_results.csv', index=False)
print("Results saved to ../data/processed/changepoint_results.csv")